Here we verify that the actual function ``expected_return_calculator.get_expected_returns`` is probably not computing the desired returns.

In [1]:
from portfolioqtopt.reader import read_welzia_stocks_file

df = read_welzia_stocks_file("/home/ggelabert/Projects/PortfolioQtOpt/data/Histórico_carteras_Welzia_2018.xlsm", "BBG (valores)",)
prices = df.to_numpy()

In [2]:
from portfolioqtopt.expand_prices import get_expand_prices

budget = 1.0
granularity_depth = 5

expand = get_expand_prices(prices, granularity_depth, budget)

In [3]:
from portfolioqtopt.expected_return_calculator import get_expected_returns

expected_returns = get_expected_returns(expand.data)  # (p,)

Now we are going to compute expected_returns in a less strange manner!

In [4]:
from portfolioqtopt.expand_prices import get_slices_list

m, n = prices.shape
a = 1 / (m-1) * (prices[-1] -prices[0]) / prices[-1]

granularity = get_slices_list(granularity_depth)
reconstruct_expected_returns = (a.reshape(-1, 1)*granularity.reshape(1, -1)).flatten()

In [5]:
import numpy as np

np.testing.assert_allclose(expected_returns, reconstruct_expected_returns)

We can conclude that surely the ``get_expected_returns`` function is not doing the wright calculus. If what we are expecting is the daily return we must rewrite this function.

The following code seems to be more correct!

In [23]:
, n = prices.shape
daily_returns = (prices[1:] - prices[:-1]) / prices[:-1]
granularity = get_slices_list(granularity_depth)
granular_daily_returns = (daily_returns.reshape(-1, 1)*granularity.reshape(1, -1)).reshape(-1, n * granularity_depth)
expected_returns = granular_daily_returns.mean(axis=0)
granular_daily_returns.shape

(374, 225)

In [24]:
m, n = prices.shape
daily_returns = ((prices[1:] - prices[:-1]) / prices[:-1]).mean(axis=0)
granularity = get_slices_list(granularity_depth)
granular_daily_returns = (daily_returns[:, np.newaxis]*granularity[np.newaxis, :]).reshape(-1, n * granularity_depth)
expected_returns_ = granular_daily_returns.mean(axis=0)

In [25]:
np.testing.assert_allclose(expected_returns, expected_returns_)